In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import time
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

generate='./data/generate/'
path='./data/rec_data/'

In [2]:
# train=pd.read_csv(generate+'train_all022.csv')
# val=pd.read_csv(generate+'test_new022.csv')

# data=pd.concat([train,val])

data=pd.read_feather(generate+'data_all023.feather')

In [3]:
lgb_cols=['pvId','suv','itemId','operator','browserType','deviceType','osType','province','city','day','category','pvid_count','pvid_item_nunique','pvid_suv_nunique','item_count','item_suv_nunique',
 'embedding_0','embedding_1','embedding_2','embedding_3','embedding_4','embedding_5','embedding_6','embedding_7','embedding_8','embedding_9','embedding_10','embedding_11','embedding_12','embedding_13','embedding_14','embedding_15','user_level',
 'length','hist_itemId0','hist_itemId1','hist_itemId2','hist_itemId3','hist_itemId4','hist_itemId5','hist_itemId6','hist_itemId7','hist_itemId8','hist_itemId9','hist_itemId10','hist_itemId11','hist_itemId12','hist_itemId13','hist_itemId14','hist_itemId15',
 'hist_itemId16','hist_itemId17','hist_itemId18','hist_itemId19','hist_itemId20',
 'hist_category0','hist_category1','hist_category2','hist_category3','hist_category4','hist_category5','hist_category6','hist_category7','hist_category8','hist_category9','hist_category10','hist_category11','hist_category12','hist_category13','hist_category14',
 'hist_category15','hist_category16','hist_category17','hist_category18','hist_category19','hist_category20','hist_item_count0','hist_item_count1','hist_item_count2','hist_item_count3','hist_item_count4','hist_item_count5','hist_item_count6','hist_item_count7','hist_item_count8','hist_item_count9','hist_item_count10',
 'hist_item_count11','hist_item_count12','hist_item_count13','hist_item_count14','hist_item_count15','hist_item_count16','hist_item_count17','hist_item_count18','hist_item_count19','hist_item_count20',
 'hist_item_level0','hist_item_level1','hist_item_level2','hist_item_level3','hist_item_level4','hist_item_level5','hist_item_level6','hist_item_level7','hist_item_level8','hist_item_level9','hist_item_level10','hist_item_level11',
 'hist_item_level12','hist_item_level13','hist_item_level14','hist_item_level15','hist_item_level16','hist_item_level17','hist_item_level18','hist_item_level19','hist_item_level20']
# ['label', 'pvId', 'suv', 'itemId', 'userSeq', 'logTs', 'operator','browserType', 'deviceType', 'osType', 'province', 'city', 'day']

In [4]:
data[['pvid_count','pvid_item_nunique','pvid_suv_nunique','item_count','item_suv_nunique']]

,pvid_count,pvid_item_nunique,pvid_suv_nunique,item_count,item_suv_nunique
0,15,15,1,6808,1
1,15,15,1,7855,1
2,15,15,1,12129,1
3,15,15,1,11587,1
4,15,15,1,9072,1
...,...,...,...,...,...
5656017,9,9,1,2201,1
5656018,9,9,1,9129,1
5656019,9,9,1,4035,1
5656020,9,9,1,972,1


In [5]:
# 数据处理sparse、dense
for feat in ['pvId','suv','user_level']:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    
train=data[data['label'].notna()]
val=data[data['label'].isna()]
del data

In [6]:
tmp=train[['pvId','pvid_count']]
tmp=tmp.drop_duplicates(keep='first')

In [7]:
g_train = tmp['pvid_count'].values.tolist()

In [8]:
len(g_train)

299919

In [9]:
# 模型及参数的定义
lgb_Classfication = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,objective='lambdarank',
                            max_depth=-1, n_estimators=10000, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2022, n_jobs= 16)

In [10]:
lgb_Classfication.fit(train[lgb_cols], train['label'],group=g_train)

LGBMRanker(colsample_bytree=0.7, learning_rate=0.01, min_child_weight=50,
           n_estimators=10000, n_jobs=16, objective='lambdarank',
           random_state=2022, reg_lambda=1, subsample=0.7, subsample_freq=1)

In [11]:
# 模型预测
val['rate'] = lgb_Classfication.predict(val[lgb_cols])

In [12]:
val['rate']

4233804    0.144468
4233805    0.131835
4233806    1.498958
4233807   -0.364411
4233808    1.111908
             ...   
5656017   -0.549430
5656018    0.638121
5656019   -0.516662
5656020   -0.186550
5656021    0.335241
Name: rate, Length: 1422218, dtype: float64

In [13]:
ygq=list(val['rate'])

# 生成提交文件
sub_path='./data/submission/'
sub=pd.read_csv(sub_path+'section2.txt', sep='\t')
sub['result'] = ygq
sub.to_csv('./data/section2DIN0.txt', sep='\t', index=False)